## Initialization
 * Import Janus Client modules
 * Setup client parameter (controller URL, user, ssh key, etc.)
 * Create a client instance
 * Create sessions for each demo job
 * Attach services to each session (host/container/profile mapping)

### Profile definition example
 * Testing with various profiles for bridge, host, sriov, and macvlan networks.
 * These, along with other profiles, are maintained on the Janus controller node
  
```yaml

```

In [55]:
%load_ext autoreload
%autoreload 2

import time
import json
from janus_client import Client, Session, Service
from ESCPeval import setup, run_job, stop_job

# Evaluation specific vars
SRC_HOST="nersc-tbn-10"
DST_HOST="nersc-dtnaas-1"
TAG="host"
ITERS=1

# Janus setup
JANUS_URL="https://nersc-srv-1.testbed100.es.net:5000"

user = 'admin'
passwd = 'admin'
keypath = '~/.ssh/id_rsa_jupyter'
mypubkey = 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC/DAAkHubwAfjOSVG67tvwvUrESRoig+OAk5htH0mLK7QaN4gTv/C3ritJH3PkRzIJyOAJVS1ssjiBtMpI8Lb8NZCYKDZdpH/803cPpd2KqI2oAWRZQcjamKYs6SlM9r380wZ069r143ldaWPU3CJ6ye2+yx0Gifdivutnk5eQ1DxdK/VbukMzWeDdO8T1QXHxqP6lRy5PUkDKlY6Ec8i8YLYijhBoMlK0NL23d9ZDrVFKYe4D9OBp0CCKzkz9a7qgLOoYvCGRzN7BXIztJkYzziFAYn4Ru0RTyHM81V8jp4H0E2EM7BDtUUBac/sk7pJS7w2rIRZtPDsFA7qFvw+1 dtnaas@a83afe25e688'

client = Client(JANUS_URL, auth=(user, passwd))

print ("\nAvailable LBNL profiles:")
print ([x['name'] for x in client.profiles().json() if x['name'].startswith("lbnl")])
print ("\n")

sess_escp = client.getSession()
srv1 = Service(instances=[DST_HOST], image='dtnaas/tools',
               profile=f'lbnl-{TAG}', username=user, public_key=mypubkey)
srv2 = Service(instances=[SRC_HOST], image='dtnaas/tools',
               profile=f'lbnl-{TAG}', username=user, public_key=mypubkey)
sess_escp.addService(srv1)
sess_escp.addService(srv2)


print (f"{sess_escp}\n")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Available LBNL profiles:
['lbnl-400g-1', 'lbnl-400g-2', 'lbnl-400g-host', 'lbnl', 'lbnl-host', 'lbnl-bridge', 'lbnl-macvlan', 'lbnl-sriov', 'lbnl-rdma']


id: e954027b-f8d0-499d-8e11-f2a0c0fdea90
allocated: False
requests: [{'instances': ['nersc-dtnaas-1'], 'image': 'dtnaas/tools', 'profile': 'lbnl-host', 'kwargs': {'USER_NAME': 'admin', 'PUBLIC_KEY': 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC/DAAkHubwAfjOSVG67tvwvUrESRoig+OAk5htH0mLK7QaN4gTv/C3ritJH3PkRzIJyOAJVS1ssjiBtMpI8Lb8NZCYKDZdpH/803cPpd2KqI2oAWRZQcjamKYs6SlM9r380wZ069r143ldaWPU3CJ6ye2+yx0Gifdivutnk5eQ1DxdK/VbukMzWeDdO8T1QXHxqP6lRy5PUkDKlY6Ec8i8YLYijhBoMlK0NL23d9ZDrVFKYe4D9OBp0CCKzkz9a7qgLOoYvCGRzN7BXIztJkYzziFAYn4Ru0RTyHM81V8jp4H0E2EM7BDtUUBac/sk7pJS7w2rIRZtPDsFA7qFvw+1 dtnaas@a83afe25e688'}}, {'instances': ['nersc-tbn-10'], 'image': 'dtnaas/tools', 'profile': 'lbnl-host', 'kwargs': {'USER_NAME': 'admin', 'PUBLIC_KEY': 'ssh-rsa AAAAB3NzaC1yc2

## DTN Service Startup
 * Start each session created in previous step
 * Check status
 * View endpoint information

In [56]:
res = sess_escp.start()
print (sess_escp.status())

id: 49, service: nersc-dtnaas-1, errors: []
id: 49, service: nersc-tbn-10, errors: []



In [57]:
print (f"-- Sess Endpoints:\n{sess_escp.endpoints()}")

-- Sess Endpoints:
nersc-dtnaas-1: nersc-dtnaas-1.testbed100.es.net:4097
nersc-tbn-10: nersc-tbn-10:4096


## Demo job setup and execution
 * ESCPeval module to initialize and run jobs given provided Janus sessions

In [78]:
# Perform some setup work on the remote containers
from ESCPeval import setup, run_job, stop_job
sess_escp.user = user
sess_escp.keypath = keypath
setup(sess_escp)

[2022-11-15 12:00:27,117] INFO: Setting up environment on nersc-dtnaas-1
[2022-11-15 12:00:32,734] INFO: Setting up environment on nersc-tbn-10


In [ ]:
from ESCPeval import setup, run_job, stop_job
# Run ESCPeval jobs
# EScp arguments will change depending on the Janus profile used for container networking
# TODO: parse this out of the Janus profile and node info
# prof = client.profiles().json().get(f"lbnl-macvlan")
# print (prof)

# Host network example below for tbn-6,7
escp_args = {"src": "10.10.2.21",
#             "src_port": 22,      # we can override EScp ssh port numbers learned from Janus session
             "dst": "10.10.2.20",
#             "dst_port": 22,      # we can override EScp ssh port numbers learned from Janus session
             "iters": 1,
             "tag": "host"}

ret = run_job(sess_escp, SRC_HOST, **escp_args)

Output()

## Janus stop and cleanup routines
 * It is possible to stop/start containers or destroy the sessions once workflow is complete

In [52]:
sess_escp.stop()

In [53]:
sess_escp.destroy()